In [31]:
# -*- coding: utf-8 -*-
%matplotlib inline
import pandas as pd
import numpy as np
import csv as csv
import pydotplus
import pyparsing

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
# from sklearn.ensemble import RandomForestClassifier

df = pd.read_csv('/home/eduzen/Projects/kaggle/titanic/data/train.csv')
df['Gender'] = df['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
df.drop(['Name', 'PassengerId', 'Fare', 'Ticket','Cabin', 'Sex'], axis=1, inplace=True)

In [32]:
# En los casos en que no figura el lugar de embarque, usamos el lugar más común
if len(df['Embarked'][ df['Embarked'].isnull() ]) > 0:
    most_common_value = df['Embarked'].dropna().mode().values
    df['Embarked'][df['Embarked'].isnull()] = most_common_value

ports = list(enumerate(np.unique(df['Embarked']))) 
ports_dict = { name : i for i, name in ports }
df.Embarked = df.Embarked.map(lambda x: ports_dict[x]).astype(int)

# Lo mismo con la edad, sino figura asignamos la edad promedio
median_age = df['Age'].dropna().median()
if len(df.Age[ df.Age.isnull() ]) > 0:
    df.Age[ df.Age.isnull() ] = median_age

/home/eduzen/.virtualenvs/kaggle-CiH6gmLr/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/eduzen/.virtualenvs/kaggle-CiH6gmLr/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [33]:
# Shape da el numero de filas y columnas
num_of_rows = df.shape[0]

# instancio un labelencoder 
label_encoder = LabelEncoder()
# instancio one hot enconder
one_encoder = OneHotEncoder()

# vamos a transformar la clase: 1°, 2° y 3° en tres columnas distitas con valores bool
new_columns_names = ['first_class', 'second_class', 'third_class']
# Usamos el label_encoder que va averiguar automaticamente cuales son todos los valores posibles
# en este caso es sencillo porque nosotros ya sabemos que son 3 valores posibles
passanger_classes = label_encoder.fit(df.Pclass) # esto da un array [1, 2, 3]  

# podria transformar los labels en numeros, en este caso no hace falta
integer_classes = passanger_classes.transform(passanger_classes.classes_).reshape(3, 1)

# Generados estos labels, transformo la columna en valores secuenciales que arrancan desde 0
# Si quiero volver a mis labels conocidos passanger_classes.inverse_transform(transformed)
transformed_passanger_classes = label_encoder.transform(df.Pclass).reshape(num_of_rows, 1) # esto da un array [0,1,2]

# con dichos valores construyo el one hot enconder que crea a partir de una row con tres valores posibles,
# un array con tres columnas.  
one_hot_encoder = one_encoder.fit(transformed_passanger_classes)
new_features = one_hot_encoder.transform(transformed_passanger_classes)
# Con este array multidimensional, voy a crear un dataframe con los nuevos nombres de columnas
new_df = pd.DataFrame(new_features.toarray(), columns=new_columns_names)

# finalmente concateno en un solo dataframe
pro_df = pd.concat([df, new_df], axis=1)

# y elimino la columna dantigua de Pclass
pro_df.drop('Pclass', axis=1, inplace=True)

In [34]:
survived = pro_df.Survived
pro_df.drop('Survived', axis=1, inplace=True)

In [35]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=3,min_samples_leaf=5)
clf = clf.fit(pro_df, survived)

In [40]:
from io import StringIO
dot_data = StringIO() 
tree.export_graphviz(clf, out_file=dot_data, feature_names=pro_df.columns)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
graph.write_png('titanic.png')

from IPython.core.display import Image 
Image(filename='titanic.png')

InvocationException: GraphViz's executables not found